In [1]:
pwd

'E:\\Projects\\MLOPS\\kidney-disease-classification-mlops\\research'

In [2]:
import os

In [3]:
os.chdir("../")

In [4]:
pwd

'E:\\Projects\\MLOPS\\kidney-disease-classification-mlops'

In [5]:
os.environ["MLFLOW_TRACKING_URI"] = ""
os.environ["MLFLOW_TRACKING_USERNAME"] = "sadhiin"
os.environ["MLFLOW_TRACKING_PASSWORD"] = ""

In [7]:
import tensorflow as tf

In [8]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [9]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [10]:
from kidneyDiseaseClassifier.constants import *
from kidneyDiseaseClassifier.utils.common import read_yaml,create_directories, save_json

In [11]:
class ConfigurationManager:
    def __init__(self, model_name= "VGG16",  config_filepath= CONFIG_FILE_PATH, params_file_path=PARAMS_FILE_PATH):
        self.model_name = model_name.upper()
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_file_path)[self.model_name]       # loading params for the specific model
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self)->EvaluationConfig:

        eval_config= EvaluationConfig(
            path_of_model= "artifacts/training/model.h5",
            training_data= "artifacts/data_ingestion/kaggle/data/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone",
            mlflow_uri= "https://dagshub.com/sadhiin/kidney-disease-classification-DL-MLOPS.mlflow",
            all_params= self.params,
            params_image_size= self.params.IMAGE_SIZE,
            params_batch_size= self.params.BATCH_SIZE
        )

        return eval_config

In [12]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

E:\Projects\MLOPS\kidney-disease-classification-mlops\venv\lib\site-packages\pydantic\_internal\_fields.py:149: UserWarning: Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
E:\Projects\MLOPS\kidney-disease-classification-mlops\venv\lib\site-packages\pydantic\_internal\_config.py:321: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


In [13]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.3
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)


        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory= self.config.training_data,
            subset='validation',
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def save_score(self):
        scores = {'loss': self.score[0], 'accuracy': self.score[1]}
        save_json(path=Path('score.json'), data= scores)

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            print(self.config.all_params)
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({'loss': self.score[0], 'accuracy':self.score[1]})

            # model registry
            if tracking_url_type_store != 'file':
                # register the model
                # There are other ways to use the model registry, which depends on the user case,
                # please refer to the doc for more information:
                # at https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, 'model', registered_model_name='VGG16')
            else:
                mlflow.keras.log_model(self.model, "model")

In [14]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(config=eval_config)

    evaluation.evaluation()
    evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2023-12-12 20:35:07,375]: INFO: common: yaml file: config\config.yaml loaded successfully!
[2023-12-12 20:35:07,409]: INFO: common: yaml file: params.yaml loaded successfully!
[2023-12-12 20:35:07,420]: INFO: common: Created directory at : artifacts
Found 3732 images belonging to 4 classes.

[2023-12-12 20:35:12,277]: WARNING: module_wrapper: From E:\Projects\MLOPS\kidney-disease-classification-mlops\venv\lib\site-packages\keras\src\utils\tf_utils.py:492: The name tf.ragged.RaggedTensorValue is deprecated. Please use tf.compat.v1.ragged.RaggedTensorValue instead.
234/234 [==============================] - 319s 1s/step - loss: 9.3505 - accuracy: 0.2980
[2023-12-12 20:40:29,683]: INFO: common: Json file saved at: score.json
{'AUGMENTATION': True, 'IMAGE_SIZE': [224, 224, 3], 'BATCH_SIZE': 16, 'INCLUDE_TOP': False, 'EPOCHS': 1, 'CLASSES': 4, 'WEIGHTS': 'imagenet', 'LEARNING_RATE': 0.01, 'DROPOUT_RATE': 0.3}


2023/12/12 20:40:32 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\Sadhin\AppData\Local\Temp\tmpswip6844\model\data\model\assets
[2023-12-12 20:40:36,395]: INFO: builder_impl: Assets written to: C:\Users\Sadhin\AppData\Local\Temp\tmpswip6844\model\data\model\assets


E:\Projects\MLOPS\kidney-disease-classification-mlops\venv\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'VGG16'.
2023/12/12 20:42:32 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16, version 1
Created version '1' of model 'VGG16'.
